In [1]:
pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.ticker as mtick

import itertools
import s3fs
import os
import torch
import random
import json
# import boto
import scipy
# import sktime
import trino

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from trino.dbapi import connect
from trino.auth import OAuth2Authentication, BasicAuthentication

from datetime import datetime
# from config import Config  
from collections import defaultdict, Counter
from tqdm import tqdm 
from itertools import zip_longest
from contextlib import closing

tqdm.pandas()

In [3]:
conn = connect(
           host=f'trino.prd.rgn',
           port=443,
           user="roan.schellingerhout@randstadgroep.nl",
           catalog='iceberg',
           verify="certificate",
           http_scheme="https",
           auth=OAuth2Authentication()
     )

In [4]:
cur = conn.cursor()

In [5]:
experience_data = cur.execute(open("./sql/candidate_work_experience.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df = pd.DataFrame(data=experience_data, columns=headers)
df.to_csv("./cleaner_data/work_experience.csv")

Open the following URL in browser for the external authentication:
https://trino.prd.rgn/oauth2/token/initiate/8328ee88553d74bd998ee61a91f547ea603ec1b5baa40b10188600453855f6bc


In [6]:
location_data = cur.execute(open("./sql/candidate_addresses.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_loc = pd.DataFrame(data=location_data, columns=headers)
df_loc.to_csv("./cleaner_data/location.csv")

In [7]:
certificate_data = cur.execute(open("./sql/candidate_certificates.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_cert = pd.DataFrame(data=certificate_data, columns=headers)

cert_map = cur.execute(open("./sql/query_certificates_map.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

cert_mapping = pd.DataFrame(data=cert_map, columns=headers)

cert_dict = cert_mapping[["request_certificate_ids", "request_certificate_description"]].set_index("request_certificate_ids").to_dict()["request_certificate_description"]
df_cert["candidate_certificate_id"] = df_cert["candidate_certificate_id"].apply(lambda x: cert_dict[x] if x in cert_dict else x)
df_cert.to_csv("./cleaner_data/certificates.csv")

In [8]:
driving_data = cur.execute(open("./sql/candidate_driving_license.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_drive = pd.DataFrame(data=driving_data, columns=headers)
df_drive.to_csv("./cleaner_data/driving_licenses.csv")

In [9]:
education_data = cur.execute(open("./sql/candidate_education.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_edu = pd.DataFrame(data=education_data, columns=headers)
df_edu.to_csv("./cleaner_data/education.csv")

In [10]:
language_data = cur.execute(open("./sql/candidate_languages.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_lang = pd.DataFrame(data=language_data, columns=headers)

lang_map = cur.execute(open("./sql/language_mapping.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

lang_mapping = pd.DataFrame(data=lang_map, columns=headers)

lang_dict = lang_mapping[["language_id", "language"]].set_index("language_id").to_dict()["language"]
df_lang["language_id"] = df_lang["language_id"].apply(lambda x: lang_dict[x])
df_lang.to_csv("./cleaner_data/languages.csv")

In [11]:
skill_data = cur.execute(open("./sql/candidate_skills.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

df_skill = pd.DataFrame(data=skill_data, columns=headers)

skill_map = cur.execute(open("./sql/skills_mapping.sql").read()).fetchall()
headers = [i[0] for i in cur.description]

skill_mapping = pd.DataFrame(data=skill_map, columns=headers)

skill_dict = skill_mapping[["skill_id", "skill_name"]].set_index("skill_id").to_dict()["skill_name"]
df_skill["skill_id"] = df_skill["skill_id"].apply(lambda x: skill_dict[x])
df_skill.to_csv("./cleaner_data/skills.csv")